In [5]:
from kafka import KafkaConsumer
from time import sleep
from json import dumps,loads
import json
from s3fs import S3FileSystem
import time
import pandas as pd

In [6]:
df = pd.DataFrame(columns=['Id', 'ActivityHour', 'Calories'])
consumer = KafkaConsumer(
    'demo_test', 
    bootstrap_servers=["localhost:9092"], 
    value_deserializer=lambda x: loads(x.decode('utf-8')),
    enable_auto_commit=False,
    auto_offset_reset='earliest',
    consumer_timeout_ms=10000
)

timeout = 10  # 10 seconds timeout if no new messages are received
start_time = time.time()

try:
    while True:
        message_batch = consumer.poll(timeout_ms=1000)
        if message_batch:
            for tp, messages in message_batch.items():
                for message in messages:

                    message_df = pd.DataFrame([message.value])
                    df = pd.concat([df, message_df], ignore_index=True)

            start_time = time.time()

        else:
            if time.time() -start_time > timeout:
                print("No new messages received for 10 seconds. Closing consumer.")
                break
finally:
    consumer.close()
print("Final DataFrame:")
print(df)


No new messages received for 10 seconds. Closing consumer.
Final DataFrame:
               Id  ActivityHour Calories
0      1503960366  3/12/16 0:00       48
1      1503960366  3/12/16 1:00       48
2      1503960366  3/12/16 2:00       48
3      1503960366  3/12/16 3:00       48
4      1503960366  3/12/16 4:00       48
...           ...           ...      ...
24079  8877689391  4/12/16 4:00       73
24080  8877689391  4/12/16 5:00       73
24081  8877689391  4/12/16 6:00       96
24082  8877689391  4/12/16 7:00      169
24083  8877689391  4/12/16 8:00      136

[24084 rows x 3 columns]
